In [ ]:
import torch
import numpy
import matplotlib.pyplot as plt

In [ ]:
print(torch.cuda.get_device_name(0))

Tesla P100-PCIE-16GB


In [ ]:
!pip install -U git+https://github.com/Adapter-Hub/adapter-transformers.git
!pip install datasets

  Cloning https://github.com/Adapter-Hub/adapter-transformers.git to /tmp/pip-req-build-fyae0_qo
  Running command git clone -q https://github.com/Adapter-Hub/adapter-transformers.git /tmp/pip-req-build-fyae0_qo
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 901kB 4.0MB/s 
     |████████████████████████████████| 3.3MB 11.7MB/s 
  Created wheel for adapter-transformers: filename=adapter_transformers-2.0.0-cp37-none-any.whl size=2097542 sha256=ca6dc32b4caba7d1d24defcc4e56920e746415360e6023b68bba92b7f941173c
  Stored in directory: /tmp/pip-ephem-wheel-cache-9pyhgxtk/wheels/b0/56/c9/5bf1c51cd513412090ad751ab10fc025210176bf0a82dd8af3
Successfully built adapter-transformers
     |████████████████████████████████| 225kB 5.3MB/s 
     |████████████████████████████████| 245kB 39.7MB/s 
     |████████████████████████████████| 112kB 22.6MB/s 


In [ ]:
import datasets
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import DataCollatorForLanguageModeling
from transformers import LineByLineTextDataset
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from transformers import Trainer, TrainingArguments
from transformers import RobertaConfig, RobertaModelWithHeads
import numpy as np
import argparse
import pdb

In [ ]:
args_dict = {
    "train_dataset": "/content/drive/MyDrive/OMSCS_DL/SCIREC/data/train.jsonl",
    "val_dataset": "/content/drive/MyDrive/OMSCS_DL/SCIREC/data/dev.jsonl",
    "test_dataset": "/content/drive/MyDrive/OMSCS_DL/SCIREC/data/test.jsonl",
    "model": "roberta-base",
    "output_dir": "/content/drive/MyDrive/OMSCS_DL/SCIREC/output/step_one/"
}

In [ ]:
# define accuracy metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="macro")
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
# loading the data
train_data = datasets.load_dataset("json",data_files=args_dict["train_dataset"])["train"]
#not a typo: load_dataset loads this as training data
val_data = datasets.load_dataset("json", data_files=args_dict["val_dataset"])["train"] 
test_data = datasets.load_dataset("json", data_files=args_dict["test_dataset"])["train"] 

Using custom data configuration default-9f081ed18e6af576


Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-9f081ed18e6af576/0.0.0/83d5b3a2f62630efc6b5315f00f20209b4ad91a00ac586597caee3a4da0bef02. Subsequent calls will reuse this data.


Using custom data configuration default-be8b4aa859efaf0f


Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-be8b4aa859efaf0f/0.0.0/83d5b3a2f62630efc6b5315f00f20209b4ad91a00ac586597caee3a4da0bef02. Subsequent calls will reuse this data.


Using custom data configuration default-3571d51e27b61b4d


Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-3571d51e27b61b4d/0.0.0/83d5b3a2f62630efc6b5315f00f20209b4ad91a00ac586597caee3a4da0bef02. Subsequent calls will reuse this data.


In [ ]:
#assign an integer key for each label
label_keys = {}
num_labels = 0
for label in train_data['label']:
    if label not in label_keys:
        label_keys[label] = num_labels
        num_labels += 1

for label in test_data['label']:
    if label not in label_keys:
        label_keys[label] = num_labels
        num_labels += 1

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

In [ ]:
def tokenization(batched_text):
    tokenized_batch = tokenizer(batched_text['text'], padding=True, truncation=True)
    tokenized_batch["label"] = [label_keys[label] for label in batched_text["label"]]
    return tokenized_batch

In [ ]:
train_data = train_data.map(tokenization, batched=True, batch_size=len(train_data))
val_data = val_data.map(tokenization, batched=True, batch_size=len(val_data))
test_data = test_data.map(tokenization, batched=True, batch_size=len(test_data))

In [ ]:
train_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
val_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

In [ ]:
f1_scores = []
for seed in range(1,4):
  #create model
  model_prev = RobertaForSequenceClassification.from_pretrained(
        args_dict["model"], hidden_dropout_prob=0.1, num_labels=num_labels)
  model_prev_configuration = model_prev.config

  model = RobertaModelWithHeads.from_pretrained(
    "roberta-base",
    config=model_prev_configuration,
  )


  adapter_name = "adapter_SCIREC_" + str(seed)
  # this part is for adding the adapter
  model.add_adapter(adapter_name)
  # Add a matching classification head
  model.add_classification_head(
      adapter_name,
      num_labels=num_labels,
    )
  # Activate the adapter
  model.train_adapter(adapter_name)
    

  training_args = TrainingArguments(
      num_train_epochs=20,
      per_device_train_batch_size=16,
      per_device_eval_batch_size=16,
      learning_rate=6e-5,
      lr_scheduler_type="constant",
      fp16=True,
      eval_accumulation_steps=20,
      save_strategy="no",
      save_steps=5000,
      save_total_limit=1,
      load_best_model_at_end=False,
      metric_for_best_model="f1",
      evaluation_strategy="epoch",
      output_dir=args_dict["output_dir"],
      overwrite_output_dir=True,
      # The next line is important to ensure the dataset labels are properly passed to the model
      remove_unused_columns=False,
      seed = seed
  )

  trainer = Trainer(
      model = model,
      args=training_args,
      compute_metrics=compute_metrics,
      train_dataset=train_data,
      eval_dataset=val_data
  )


  trainer.train()

  metrics = trainer.evaluate(test_data)
  print("Metrics for seed {}: {}".format(seed, metrics))

  f1_scores.append(metrics["eval_f1"])

  model.save_adapter(args_dict["output_dir"] + adapter_name, adapter_name, with_head=True)




Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Runtime,Samples Per Second
1,No log,1.401121,0.509890,0.174906,0.378534,0.210885,3.185700,142.826000
2,No log,0.990338,0.661538,0.397528,0.532761,0.421944,3.109400,146.330000
3,1.265400,0.692153,0.756044,0.574803,0.647227,0.576853,3.067100,148.348000
4,1.265400,0.554994,0.804396,0.710731,0.767391,0.710204,3.143800,144.728000
5,0.662500,0.517739,0.826374,0.766222,0.787677,0.770933,3.118100,145.922000
6,0.662500,0.461059,0.832967,0.745409,0.801787,0.726691,3.091400,147.184000
7,0.662500,0.485561,0.835165,0.765877,0.803984,0.777733,3.096000,146.961000
8,0.444300,0.460657,0.837363,0.781685,0.776156,0.809184,3.170800,143.495000
9,0.444300,0.512820,0.819780,0.752576,0.772967,0.778239,3.054900,148.939000
10,0.340300,0.378530,0.861538,0.812883,0.813506,0.816214,3.074900,147.973000


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Metrics for seed 1: {'eval_loss': 0.7553009986877441, 'eval_accuracy': 0.8593429158110883, 'eval_f1': 0.7975383502761327, 'eval_precision': 0.7715022139714004, 'eval_recall': 0.8282392930406596, 'eval_runtime': 6.3168, 'eval_samples_per_second': 154.193, 'epoch': 20.0, 'eval_mem_cpu_alloc_delta': -4096, 'eval_mem_gpu_alloc_delta': -7168, 'eval_mem_cpu_peaked_delta': 4096, 'eval_mem_gpu_peaked_delta': 129381376}


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Runtime,Samples Per Second
1,No log,1.539624,0.470330,0.091394,0.067190,0.142857,3.057600,148.812000
2,No log,1.085893,0.613187,0.289196,0.387291,0.332913,3.003900,151.469000
3,1.314000,0.910505,0.659341,0.496614,0.593954,0.516907,3.113200,146.153000
4,1.314000,0.647571,0.760440,0.657331,0.713728,0.638915,3.064500,148.475000
5,0.735700,0.562008,0.824176,0.749189,0.802997,0.726949,3.083400,147.563000
6,0.735700,0.525556,0.815385,0.779467,0.782493,0.792047,3.073800,148.027000
7,0.735700,0.440359,0.846154,0.794386,0.800145,0.800497,3.094200,147.048000
8,0.468100,0.428277,0.854945,0.800213,0.832788,0.794332,3.085500,147.462000
9,0.468100,0.415449,0.859341,0.811048,0.812932,0.818301,3.038700,149.736000
10,0.350800,0.396575,0.854945,0.800823,0.815968,0.790996,3.061500,148.622000


Metrics for seed 2: {'eval_loss': 0.5342245101928711, 'eval_accuracy': 0.8788501026694046, 'eval_f1': 0.7999390021497644, 'eval_precision': 0.8204864963471559, 'eval_recall': 0.7880666763550875, 'eval_runtime': 6.4758, 'eval_samples_per_second': 150.407, 'epoch': 20.0, 'eval_mem_cpu_alloc_delta': 0, 'eval_mem_gpu_alloc_delta': 0, 'eval_mem_cpu_peaked_delta': 0, 'eval_mem_gpu_peaked_delta': 129375232}


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Runtime,Samples Per Second
1,No log,1.509897,0.470330,0.091394,0.067190,0.142857,3.132400,145.256000
2,No log,1.087615,0.613187,0.308954,0.408603,0.329936,3.106800,146.451000
3,1.285400,0.787068,0.703297,0.519573,0.579296,0.500857,3.074200,148.005000
4,1.285400,0.589929,0.780220,0.667178,0.721301,0.647664,3.096300,146.950000
5,0.703700,0.536041,0.806593,0.702864,0.786452,0.674877,3.031500,150.092000
6,0.703700,0.461935,0.837363,0.760582,0.796139,0.756982,3.052800,149.043000
7,0.703700,0.495075,0.839560,0.775130,0.797674,0.796466,3.061600,148.615000
8,0.437400,0.417664,0.850549,0.796002,0.799509,0.797812,2.998500,151.740000
9,0.437400,0.431548,0.861538,0.812309,0.821883,0.823122,3.040500,149.644000
10,0.341200,0.402638,0.865934,0.808387,0.822101,0.807665,3.057100,148.832000


Metrics for seed 3: {'eval_loss': 0.6085906624794006, 'eval_accuracy': 0.87782340862423, 'eval_f1': 0.8104807179960051, 'eval_precision': 0.8122515137816656, 'eval_recall': 0.8164694267854689, 'eval_runtime': 6.559, 'eval_samples_per_second': 148.499, 'epoch': 20.0, 'eval_mem_cpu_alloc_delta': 0, 'eval_mem_gpu_alloc_delta': 0, 'eval_mem_cpu_peaked_delta': 0, 'eval_mem_gpu_peaked_delta': 135697920}


In [ ]:
print("Avg f1 score: {}, stdev: {}".format(np.mean(f1_scores), np.std(f1_scores)))

Avg f1 score: 0.8026526901406341, stdev: 0.005621346068510696


In [ ]:
f1_scores

[0.7817573034330654, 0.7780967346908227, 0.7880080707098019]

In [ ]:
# testing the inference logic:
inf_model = RobertaModelWithHeads.from_pretrained(
    "roberta-base",
    config=model_prev_configuration,
)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModelWithHeads were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.embeddings.position_ids', 'roberta.encoder.layer.0.output.adapters.adapter_SCIREC_1.adapter_down.0.weight', 'roberta.encoder.layer.0

In [ ]:
inf_model.load_adapter(args_dict["output_dir"] + adapter_name)

Overwriting existing adapter 'adapter_SCIREC_1'.
Overwriting existing head 'adapter_SCIREC_1'


'adapter_SCIREC_1'

In [ ]:
trainer_eval = Trainer(
    model=inf_model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=val_data
)


In [ ]:
metrics = trainer_eval.evaluate(test_data)
print("Metrics for seed {}: {}".format(seed, metrics))

f1_scores.append(metrics["eval_f1"])

print("Avg f1 score: {}, stdev: {}".format(np.mean(f1_scores), np.std(f1_scores)))

Metrics for seed 1: {'eval_loss': 1.989552617073059, 'eval_accuracy': 0.060574948665297744, 'eval_f1': 0.016318628128889505, 'eval_precision': 0.008653564095042535, 'eval_recall': 0.14285714285714285, 'eval_runtime': 9.6565, 'eval_samples_per_second': 100.865, 'init_mem_cpu_alloc_delta': 0, 'init_mem_gpu_alloc_delta': 505846784, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'eval_mem_cpu_alloc_delta': 0, 'eval_mem_gpu_alloc_delta': 0, 'eval_mem_cpu_peaked_delta': 0, 'eval_mem_gpu_peaked_delta': 147916800}
Avg f1 score: 0.37531543413252044, stdev: 0.35899680600363093


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sat May  1 15:23:05 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    33W /  70W |   4446MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------